In [1]:
#MCDA Analysis with Wind and Solar Data. No Normalization or Reclassification of Dataset. Wind Priortized
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np

def scale_column(df, column_name, invert=False):
    min_val = df[column_name].min()
    max_val = df[column_name].max()
    if invert:
        df[f"{column_name}_scaled"] = 1 - (df[column_name] - min_val) / (max_val - min_val)
    else:
        df[f"{column_name}_scaled"] = (df[column_name] - min_val) / (max_val - min_val)
    return df

# === Input file paths ===
SOLAR_FILE = r"C:\GISDataManipulation\Merged_Denmark_Norway_SolarPVOUT\Merged_SolarPVOUT_WithAverage.csv"
WIND_FILE = r"C:\GISDataManipulation\NorwayDenmarkMergedWindSpeed100m\NorwayDenmarkMergedWindSpeed100mFromTXT.txt"
MCDA_OUTPUT_FILE = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\MCDA_Wind_Solar_Combined_Score_WindP.csv"

# === Load and scale data ===
df_solar = pd.read_csv(SOLAR_FILE)
df_wind = pd.read_csv(WIND_FILE)

df_solar = scale_column(df_solar, 'PVOUT_Average', invert=False)
df_wind = scale_column(df_wind, 'Wind_Speed', invert=False)

# === Perform nearest-neighbor join using KDTree ===
solar_coords = df_solar[['Latitude', 'Longitude']].values
wind_coords = df_wind[['Latitude', 'Longitude']].values

tree = cKDTree(wind_coords)
distances, indices = tree.query(solar_coords, k=1, distance_upper_bound=0.001)  # ~100m tolerance

# Filter matched results
valid_mask = (indices < len(df_wind)) & (np.isfinite(distances))
df_solar_valid = df_solar[valid_mask].reset_index(drop=True)
matched_wind = df_wind.iloc[indices[valid_mask]].reset_index(drop=True)

# Combine matched data
df_combined = df_solar_valid.copy()
df_combined['Wind_Speed_scaled'] = matched_wind['Wind_Speed_scaled'].values

# === Assign weights and compute MCDA score ===
WIND_WEIGHT = 0.6
SOLAR_WEIGHT = 0.4

df_combined['MCDA_Score'] = (
    df_combined['Wind_Speed_scaled'] * WIND_WEIGHT +
    df_combined['PVOUT_Average_scaled'] * SOLAR_WEIGHT
)

# Save output
df_combined[['Latitude', 'Longitude', 'MCDA_Score']].to_csv(MCDA_OUTPUT_FILE, index=False)
print(f"MCDA combined wind+solar score saved at: {MCDA_OUTPUT_FILE}")

# Show top 5 locations
print("Top 5 best locations based on MCDA score:")
print(df_combined[['Latitude', 'Longitude', 'MCDA_Score']].sort_values(by='MCDA_Score', ascending=False).head(5))


MCDA combined wind+solar score saved at: C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\MCDA_Wind_Solar_Combined_Score_WindP.csv
Top 5 best locations based on MCDA score:
         Latitude  Longitude  MCDA_Score
124473  59.904167   7.162500    0.747381
126213  59.937500   7.120833    0.740130
126798  59.954167   7.112500    0.737280
124474  59.904167   7.170833    0.734824
126212  59.937500   7.112500    0.730284


In [6]:
#MCDA Analysis with Wind and Solar Data. No Normalization or Reclassification of Dataset. Wind Priortized Plotting
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster

# Input MCDA CSV file
MCDA_FILE = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\MCDA_Wind_Solar_Combined_Score_WindP.csv"
OUTPUT_TOP5_CLUSTERED_LOCATIONS = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\WindPOutputFilesMCDA_Top5_Clustered_Best_Locations.csv"
OUTPUT_TOP5_MAP = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\MCDA_Top5_Clustered_Map_WindP.html"

# Load MCDA data
df = pd.read_csv(MCDA_FILE)

# Perform DBSCAN clustering (proximity tolerance, e.g., eps=0.01 ~1km)
coords = df[['Latitude', 'Longitude']].values
clustering = DBSCAN(eps=0.01, min_samples=1).fit(coords)
df['Cluster'] = clustering.labels_

# Select the best location from each cluster (highest MCDA score)
best_locations = []
for cluster_id in df['Cluster'].unique():
    cluster_data = df[df['Cluster'] == cluster_id]
    best_location = cluster_data.sort_values(by='MCDA_Score', ascending=False).iloc[0].copy()
    best_location['Cluster_Size'] = len(cluster_data)
    best_locations.append(best_location)

df_best = pd.DataFrame(best_locations)

# Sort by MCDA_Score and select top 5 locations
df_top5 = df_best.sort_values(by='MCDA_Score', ascending=False).head(5)
df_top5.to_csv(OUTPUT_TOP5_CLUSTERED_LOCATIONS, index=False)
print(f"Top 5 clustered best locations saved to: {OUTPUT_TOP5_CLUSTERED_LOCATIONS}")

# Create Folium map
center_lat = df_top5["Latitude"].mean()
center_lon = df_top5["Longitude"].mean()

folium_map = folium.Map(location=[center_lat, center_lon], zoom_start=6)
marker_cluster = MarkerCluster().add_to(folium_map)

# Add top 5 locations as markers
for _, row in df_top5.iterrows():
    popup_text = f"""
    <b>Latitude:</b> {row['Latitude']}<br>
    <b>Longitude:</b> {row['Longitude']}<br>
    <b>MCDA Score:</b> {row['MCDA_Score']:.4f}

    """
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=9,
        color="green",
        fill=True,
        fill_opacity=0.9,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(marker_cluster)

# Save the top 5 map
folium_map.save(OUTPUT_TOP5_MAP)
print(f"Top 5 clustered locations map saved to: {OUTPUT_TOP5_MAP}")


Top 5 clustered best locations saved to: C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\WindPOutputFilesMCDA_Top5_Clustered_Best_Locations.csv
Top 5 clustered locations map saved to: C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\MCDA_Top5_Clustered_Map_WindP.html


In [4]:
#MCDA Analysis with Wind and Solar Data. No Normalization or Reclassification of Dataset. Solar Priortized
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np

def scale_column(df, column_name, invert=False):
    min_val = df[column_name].min()
    max_val = df[column_name].max()
    if invert:
        df[f"{column_name}_scaled"] = 1 - (df[column_name] - min_val) / (max_val - min_val)
    else:
        df[f"{column_name}_scaled"] = (df[column_name] - min_val) / (max_val - min_val)
    return df

# === Input file paths ===
SOLAR_FILE = r"C:\GISDataManipulation\Merged_Denmark_Norway_SolarPVOUT\Merged_SolarPVOUT_WithAverage.csv"
WIND_FILE = r"C:\GISDataManipulation\NorwayDenmarkMergedWindSpeed100m\NorwayDenmarkMergedWindSpeed100mFromTXT.txt"
MCDA_OUTPUT_FILE = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\MCDA_Wind_Solar_Combined_Score_SolarP.csv"

# === Load and scale data ===
df_solar = pd.read_csv(SOLAR_FILE)
df_wind = pd.read_csv(WIND_FILE)

df_solar = scale_column(df_solar, 'PVOUT_Average', invert=False)
df_wind = scale_column(df_wind, 'Wind_Speed', invert=False)

# === Perform nearest-neighbor join using KDTree ===
solar_coords = df_solar[['Latitude', 'Longitude']].values
wind_coords = df_wind[['Latitude', 'Longitude']].values

tree = cKDTree(wind_coords)
distances, indices = tree.query(solar_coords, k=1, distance_upper_bound=0.001)  # ~100m tolerance

# Filter matched results
valid_mask = (indices < len(df_wind)) & (np.isfinite(distances))
df_solar_valid = df_solar[valid_mask].reset_index(drop=True)
matched_wind = df_wind.iloc[indices[valid_mask]].reset_index(drop=True)

# Combine matched data
df_combined = df_solar_valid.copy()
df_combined['Wind_Speed_scaled'] = matched_wind['Wind_Speed_scaled'].values

# === Assign weights and compute MCDA score ===
WIND_WEIGHT = 0.4
SOLAR_WEIGHT = 0.6

df_combined['MCDA_Score'] = (
    df_combined['Wind_Speed_scaled'] * WIND_WEIGHT +
    df_combined['PVOUT_Average_scaled'] * SOLAR_WEIGHT
)

# Save output
df_combined[['Latitude', 'Longitude', 'MCDA_Score']].to_csv(MCDA_OUTPUT_FILE, index=False)
print(f"MCDA combined wind+solar score saved at: {MCDA_OUTPUT_FILE}")

# Show top 5 locations
print("Top 5 best locations based on MCDA score:")
print(df_combined[['Latitude', 'Longitude', 'MCDA_Score']].sort_values(by='MCDA_Score', ascending=False).head(5))


MCDA combined wind+solar score saved at: C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\MCDA_Wind_Solar_Combined_Score_SolarP.csv
Top 5 best locations based on MCDA score:
         Latitude  Longitude  MCDA_Score
124473  59.904167   7.162500    0.826160
126213  59.937500   7.120833    0.821543
126798  59.954167   7.112500    0.817318
126212  59.937500   7.112500    0.813181
124474  59.904167   7.170833    0.810792


In [7]:
#MCDA Analysis with Wind and Solar Data. No Normalization or Reclassification of Dataset. Solar Priortized Plotting
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import MarkerCluster

# Input MCDA CSV file
MCDA_FILE = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\MCDA_Wind_Solar_Combined_Score_SolarP.csv"
OUTPUT_TOP5_CLUSTERED_LOCATIONS = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\SolarPOutputFilesMCDA_Top5_Clustered_Best_Locations.csv"
OUTPUT_TOP5_MAP = r"C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\MCDA_Top5_Clustered_Map_SolarP.html"

# Load MCDA data
df = pd.read_csv(MCDA_FILE)

# Perform DBSCAN clustering (proximity tolerance, e.g., eps=0.01 ~1km)
coords = df[['Latitude', 'Longitude']].values
clustering = DBSCAN(eps=0.01, min_samples=1).fit(coords)
df['Cluster'] = clustering.labels_

# Select the best location from each cluster (highest MCDA score)
best_locations = []
for cluster_id in df['Cluster'].unique():
    cluster_data = df[df['Cluster'] == cluster_id]
    best_location = cluster_data.sort_values(by='MCDA_Score', ascending=False).iloc[0].copy()
    best_location['Cluster_Size'] = len(cluster_data)
    best_locations.append(best_location)

df_best = pd.DataFrame(best_locations)

# Sort by MCDA_Score and select top 5 locations
df_top5 = df_best.sort_values(by='MCDA_Score', ascending=False).head(5)
df_top5.to_csv(OUTPUT_TOP5_CLUSTERED_LOCATIONS, index=False)
print(f"Top 5 clustered best locations saved to: {OUTPUT_TOP5_CLUSTERED_LOCATIONS}")

# Create Folium map
center_lat = df_top5["Latitude"].mean()
center_lon = df_top5["Longitude"].mean()

folium_map = folium.Map(location=[center_lat, center_lon], zoom_start=6)
marker_cluster = MarkerCluster().add_to(folium_map)

# Add top 5 locations as markers
for _, row in df_top5.iterrows():
    popup_text = f"""
    <b>Latitude:</b> {row['Latitude']}<br>
    <b>Longitude:</b> {row['Longitude']}<br>
    <b>MCDA Score:</b> {row['MCDA_Score']:.4f}
    """
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=9,
        color="green",
        fill=True,
        fill_opacity=0.9,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(marker_cluster)

# Save the top 5 map
folium_map.save(OUTPUT_TOP5_MAP)
print(f"Top 5 clustered locations map saved to: {OUTPUT_TOP5_MAP}")


Top 5 clustered best locations saved to: C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\SolarPOutputFilesMCDA_Top5_Clustered_Best_Locations.csv
Top 5 clustered locations map saved to: C:\GISDataManipulation\MCDA Files\SolarandWindMCDA\OutputFiles\MCDA_Top5_Clustered_Map_SolarP.html
